#Bienvenidos al taller:
# Aceleración de procesos
# en CPU/GPU

Mini Workshop on
High Performance Computing in Science and Engineering 











Noviembre 2022.

Scalar vs Vector

 En este caso, (4 valorea por vector) (calsico)

# Introducción a SIMD

 SSE, AVX2, AVX512, etc, without modifications.

<img src="https://github.com/OscarHuesos/Taller_W_HPCSE-aceleracion/blob/main/simd.png?raw=true:, width=600" alt="simd" width=600>


In [ ]:
!git clone https://github.com/OscarHuesos/Taller_W_HPCSE-aceleracion.git

Cloning into 'Taller_W_HPCSE-aceleracion'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 29 (delta 6), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (29/29), done.


In [ ]:
!apt-get update
!apt install -y cmake
!pip install dlib

!apt-get install git dpkg-dev binutils libx11-dev libxpm-dev libxft-dev libxext-dev
!apt-get install git tar gfortran subversion

In [ ]:
 %%bash
 gcc -mavx2 -dM -E - < /dev/null | egrep "SSE|AVX" | sort

#define __AVX__ 1
#define __AVX2__ 1
#define __SSE__ 1
#define __SSE2__ 1
#define __SSE2_MATH__ 1
#define __SSE3__ 1
#define __SSE4_1__ 1
#define __SSE4_2__ 1
#define __SSE_MATH__ 1
#define __SSSE3__ 1


Ventajas de aplicaciones 

In [ ]:
!git clone https://github.com/VcDevel/Vc.git

Cloning into 'Vc'...
remote: Enumerating objects: 37349, done.
remote: Counting objects: 100% (386/386), done.
remote: Compressing objects: 100% (195/195), done.
remote: Total 37349 (delta 231), reused 277 (delta 183), pack-reused 36963
Receiving objects: 100% (37349/37349), 11.23 MiB | 21.42 MiB/s, done.
Resolving deltas: 100% (28558/28558), done.


In [ ]:
%%bash
cd Vc
git tag -l
git checkout -b 1.3.3 1.3.3
mkdir build
mkdir install

In [ ]:
%%bash
cd Vc
cd build
cmake -DCMAKE_INSTALL_PREFIX=/content/Vc/install  /content/Vc 
make -j4
make install

In [ ]:
!git clone https://gitlab.cern.ch/VecGeom/VecCore

Cloning into 'VecCore'...
remote: Enumerating objects: 4636, done.
remote: Counting objects: 100% (4636/4636), done.
remote: Compressing objects: 100% (1865/1865), done.
remote: Total 4636 (delta 2676), reused 4586 (delta 2631), pack-reused 0
Receiving objects: 100% (4636/4636), 14.54 MiB | 12.23 MiB/s, done.
Resolving deltas: 100% (2676/2676), done.


rrtghb

In [ ]:
%%bash
cd VecCore
mkdir build
mkdir install

In [ ]:
%%bash
cd VecCore
cd build
cmake -DCMAKE_INSTALL_PREFIX=/content/VecCore/install -DVc_DIR=/content/Vc/install/lib/cmake/Vc   \
-DVECCORE_ENABLE_VC=ON -DBACKEND=Vc -DVC=ON /content/VecCore
make -j4

In [ ]:
%%bash
cd VecCore
cd build
make install

root

<img src="https://github.com/OscarHuesos/Taller_W_HPCSE-aceleracion/blob/main/maskops.png?raw=true:, width=600" alt="simd" width=600>

overrite same function en funcion of vector backend

In [ ]:
%%writefile example_uno.cpp
#include <cstdio>
#include <cstdlib>
#include <string>
#include <chrono>
#include <cstddef>
#include <cstdint>
#include <math.h>
#include <VecCore/VecCore>
#include <Vc/Vc>
#include "VecCore/Backend/VcVector.h"

using namespace vecCore;


template<typename T>
T Counter(){
T w=0.002;
T u1= 0.9;
T u2= 12;
return math::Cos(u1)+math::Exp(w)+math::Sqrt(u2);
}


template<typename T>
void Acumulator(int n){

size_t size =  VectorSize<T>();
printf("size %d \n", size);
T counter;
float count=0;
float s;


for(int i=0; i< n/size; ++i){

counter = Counter<T>();
for (int j=0; j< size ; ++j) {
s = Get(counter,j );  
count = count + s;
}  

}

printf("Acumulador %f \n", count);
return;
}


int main(){

printf("Primer ejemplo \n");

using VectorBackend = backend::VcVector;
using ScalarBackend = backend::Scalar;

int N=100;

auto startscalar = std::chrono::system_clock::now();
Acumulator<backend::Scalar::Float_v>(N);   
auto endscalar = std::chrono::system_clock::now();

std::chrono::duration<float,std::milli> durationscalar = endscalar - startscalar;
std::cout << durationscalar.count() << " ms" << std::endl;

auto startvector = std::chrono::system_clock::now();
Acumulator<backend::VcVector::Float_v>(N); 
auto endvector = std::chrono::system_clock::now();

std::chrono::duration<float,std::milli> durationvector = endvector - startvector;
std::cout << durationvector.count() << " ms" << std::endl;

return 0;
}

Writing example_uno.cpp


Mencionar problema latencia

In [ ]:
%%writefile example_dos.cpp
#include <cstdio>
#include <cstdlib>
#include <string>
#include <bits/stdc++.h>
#include <random>
#include <algorithm>

#include <VecCore/VecCore>
#include <Vc/Vc>
#include <VecCore/Types.h>
#include "VecCore/Backend/VcVector.h"
#include <math.h>
#include <chrono>

using namespace vecCore;


void histograma (float *arr, int nsamples, float low, float high, int bins){
int data[bins];
int histo[bins];
float paso = (high-low)/bins;

printf("paso %f \n", paso);

float r_inf;
float r_sup;
int cont=0;
float conv_factor = 0.01*nsamples;


for (int i=0; i< bins; ++i) {

r_inf = low+paso*i; 
r_sup = low+paso*(i+1);

//printf("val inf %f \n", r_inf);
//printf("val sup %f \n", r_sup);

for (int j=0; j< nsamples; ++j) {
if(   (arr[j] > r_inf )  && (arr[j]  < r_sup )      ){ 
cont=cont+1;
}

}
data[i]=cont;
cont=0;
}

for (int k=0; k< bins; ++k) {
printf("bins %d \n", data[k]);
}


for (int kk=0; kk< bins; ++kk) {
histo[kk] = data[kk]/conv_factor;

r_inf = low+paso*kk; 
r_sup = low+paso*(kk+1);


printf("Intervalo (%f a %f ) ", r_inf, r_sup );

 for (int jj=0; jj < histo[kk]; jj++) {
  printf("*");
   }
printf(" \n");
}

return;
}



template <typename T>
T Gauss_distribution(T u1,T u2, T mean, T sigma, T pi){
T Norm=  math::Sqrt(- (math::Log(u1) + math::Log(u1)) );
return mean+sigma*(math::Cos(pi*(u2+u2))*Norm);
}


template <typename T>
void VectorGauss(int nsamples, float *array ){

size_t size =  VectorSize<T>();
printf("size %d \n", size);

T res=0;
std::default_random_engine generator;
std::uniform_real_distribution<float> distribution(0.0,1.0);
T m= 0.0;
T p=3.1416;
T desv=1.0;
T numberu1;
T numberu2;

for (int i=0; i<nsamples/size ; ++i) {
numberu1 = distribution(generator);
numberu2 = distribution(generator);

res=Gauss_distribution<T>(numberu1, numberu2, m, desv, p);

for (int j=0; j< size ; ++j) {
array[i*size+j] = res[j];
}  


}


return;
}



template <typename T>
void ScalarGauss(int nsamples, float* array ){

size_t size =  VectorSize<T>();
printf("size %d \n", size);

std::default_random_engine generator;
std::uniform_real_distribution<T> distribution(0.0,1.0);
T mean = 0.0;
T p=3.1416;
T desv=1.0;
T r=0;
T numberu1;
T numberu2;

for (int i=0; i<nsamples; ++i) {
numberu1 = distribution(generator);
numberu2 = distribution(generator);
array[i] = Gauss_distribution<T>(numberu1, numberu2, mean, desv, p);
}

return;
}


int main(){


int nsamples=1000000;
printf("main dos \n");

float array_scalar[nsamples];
float array_vector[nsamples];

using VectorBackend = backend::VcVector;
using ScalarBackend = backend::Scalar;

auto startscalar = std::chrono::system_clock::now();
ScalarGauss<backend::Scalar::Float_v>(nsamples, array_scalar);   
auto endscalar = std::chrono::system_clock::now();

std::chrono::duration<float,std::milli> durationscalar = endscalar - startscalar;
std::cout << durationscalar.count() << " ms" << std::endl;

auto startvector = std::chrono::system_clock::now();
VectorGauss<backend::VcVector::Float_v>(nsamples, array_vector); 
auto endvector = std::chrono::system_clock::now();

std::chrono::duration<float,std::milli> durationvector = endvector - startvector;
std::cout << durationvector.count() << " ms" << std::endl;

float bajo = -4;
float alto = 4;
int bins=14;

std::cout << "Histograma Distribucion Gaussiana Scalar" << std::endl;
histograma(array_scalar, nsamples,bajo, alto, bins );

std::cout << "Histograma Distribucion Gaussiana Vector" << std::endl;
histograma(array_vector, nsamples,bajo, alto, bins );


return 0;
}

Overwriting example_dos.cpp


 anterior same 4 values for each entry

<img src="https://github.com/OscarHuesos/Taller_W_HPCSE-aceleracion/blob/main/%20Normalvec.jpg?raw=true:, width=600" alt="norm" width=600>

In [ ]:
%%writefile example_tres.cpp
#include <cstdio>
#include <cstdlib>
#include <string>
#include <chrono>
#include <random>
#include <math.h>

#include <VecCore/Types.h>
#include <Vc/Vc>
#include <VecCore/VecCore>
#include "VecCore/Backend/VcVector.h"


using namespace vecCore;

template <typename T>
T Rho (T x , T y ){
return math::Sqrt(x*x+y*y);
}  

template <typename T>
T Theta (T x , T y ){
return math::ATan(y/x);
}  


template <typename T>
void VectorCartesian (T* Xv , T* Yv , float* R,  float* The,  size_t n){

size_t size =  VectorSize<T>();
printf("size %d \n", size);
T r;
T ang;

for (int i=0; i< n ; ++i) {

r   =  Rho<T> (Xv[i] , Yv[i] );
ang =  Theta<T>(Xv[i] , Yv[i] );

for (int j=0; j< size ; ++j) {
R[i*size+j] = r[j];
The[i*size+j] = ang[j];
}  

}
 
return;
} 


template <typename T>
void ScalarCartesian (float* X , float* Y , float* R,  float* The,  int n){

size_t size =  VectorSize<T>();
printf("size %d \n", size);
T r;
T ang;


for (int i=0; i< n ; ++i) {

r   =  Rho<T> (X[i] , Y[i] );
ang =  Theta<T>(X[i] , Y[i] );

R[i]=r;
The[i]=ang;
}
 
return;
}  





int main(){

printf("Ejemplo tres \n");

int N=100000;
float array_scalar_R[N];
float array_vector_R[N];

float array_scalar_T[N];
float array_vector_T[N];

float X_scalar[N];
float Y_scalar[N];

using VectorBackend = backend::VcVector;
using ScalarBackend = backend::Scalar;

using Float_v = typename backend::VcVector::Float_v;

size_t size =  VectorSize<backend::VcVector::Float_v>();
size_t sizeV=  N/size;

Float_v X_vector[sizeV];
Float_v Y_vector[sizeV];

std::default_random_engine generator;
std::uniform_real_distribution<float> distribution(0.0,1.0);

for (int i=0; i< N; ++i) {
X_scalar[i] = distribution(generator);
Y_scalar[i] = distribution(generator);
}


for (int j=0; j < sizeV; ++j) {
for(int k=0; k < size; ++k){
Set( X_vector[j] , k ,   X_scalar[j*size+k]  );
Set( Y_vector[j] , k ,   Y_scalar[j*size+k]  );
}
}

auto startscalar = std::chrono::system_clock::now();
ScalarCartesian<backend::Scalar::Float_v>( X_scalar, Y_scalar, array_scalar_R, array_scalar_T,  N);   
auto endscalar = std::chrono::system_clock::now();

std::chrono::duration<float,std::milli> durationscalar = endscalar - startscalar;
std::cout << durationscalar.count() << " ms" << std::endl;

auto startvector = std::chrono::system_clock::now();
VectorCartesian<backend::VcVector::Float_v>( X_vector , Y_vector , array_vector_R, array_vector_T, sizeV );
auto endvector = std::chrono::system_clock::now();

std::chrono::duration<float,std::milli> durationvector = endvector - startvector;
std::cout << durationvector.count() << " ms" << std::endl;


float r_scalar=0;
float t_scalar=0;

float r_vector=0;
float t_vector=0;

for (int l=0; l< N; ++l) {
r_scalar = r_scalar + array_scalar_R[l];
t_scalar = t_scalar + array_scalar_T[l];

r_vector = r_vector + array_vector_R[l];
t_vector = t_vector + array_vector_T[l];

}

printf("rho scalar %f theta scalar %f rho vector %f theta vector %f \n", r_scalar, t_scalar, r_vector, t_vector  );

return 0;
}

Overwriting example_tres.cpp


Suma de función de distribución de probabilidad exponencial:


\begin{equation}
S= \sum_{n=0,4}^{N} 
\end{equation}


Vamos a aproximar fijando limites arbritrarios y $N$:

\begin{equation}
S \approx \sum_{n=1}^{1000, 000}  \int_{0}^{ 1000 } x n e^{-nx}  dx
\end{equation}


Naive Simspon 3/8:

In [ ]:
%%writefile example_cuatro.cpp
#include <cstdio>
#include <cstdlib>
#include <string>
#include <random>
#include <cfloat>

#include <VecCore/VecCore>
#include <Vc/Vc>
#include <VecCore/Types.h>
#include "VecCore/Backend/VcVector.h"
#include <math.h>
#include <chrono>


using namespace vecCore;
using VectorBackend = backend::VcVector;
using ScalarBackend = backend::Scalar;

using Double_v = typename backend::VcVector::Double_v;

template <typename T>
T acum_function_scalar (T x , int n ){

T ac= T(0);

if(  ( x > 0)  &&  (x < 1)  ) {
for(int i=0 ; i<n ; ++i){
ac= ac +  math::Exp(-i*x);
}
}
return ac;
} 

template <typename T>
T acum_function_vector (T x , int n ){

T ac= T(0);
Mask<T> gtOne (x > 0);
Mask<T> gtTwo (x < 1);
if(  MaskFull(gtOne) &&  MaskFull(gtTwo)    ) {

for(int i=0 ; i <n ; ++i){
ac= ac +  math::Exp(-i*x);
}
}
return ac;
}  


template <typename T>
double VectorFun (T* vec, int N ){

size_t size  =  VectorSize<backend::VcVector::Double_v>();
size_t sizeV =  N/size;

double res=0;
T vcc = T(0); 

for (int i=0; i < sizeV; ++i) {

vcc = acum_function_vector (vec[i], i);

for (int j=0; j < size; ++j) {

if(!std::isnan(vcc[j])){
res= res + vcc[j];
}else{

}

}  
}

return res;
  
}

  

template <typename T>
double ScalarFun (T* vc, int N ){

size_t size =  VectorSize<backend::VcVector::Double_v>();
size_t sizeV=  N/size;

double res=0;
double r;

for (int i=0; i < sizeV; ++i) {
for(int k=0; k < size; ++k){
r= acum_function_scalar( vc[i*size+k] ,i);
if(!std::isnan(r)  ){
res= res + r;
}else{

}

}
}

return res;

}

  
int main(){

int N= 40000 ;
printf("Ejemplo cuatro \n");

size_t size =  VectorSize<backend::VcVector::Double_v>();
size_t sizeV=  N/size;

double array_scalar[N];
double array_vector[N];

double r_scalar;
double r_vector;

double ran_scalar[N];
Double_v ran_vector[sizeV];

std::default_random_engine generator;
std::uniform_real_distribution<double> distribution(0.0,1.0);

double r;

for (int j=0; j < sizeV; ++j) {
for(int k=0; k < size; ++k){
r = distribution(generator);
Set( ran_vector[j] , k ,   r );
ran_scalar[j*size+k] = r;
}
}

auto startscalar = std::chrono::system_clock::now();
r_scalar = ScalarFun<backend::Scalar::Double_v>(ran_scalar, N );   
auto endscalar = std::chrono::system_clock::now();

std::chrono::duration< float,std::milli> durationscalar = endscalar - startscalar;
std::cout << "Tiempo Scalar " << durationscalar.count() << " ms" << std::endl;
std::cout << "Suma Scalar  sum: " << r_scalar << std::endl;

auto startvector = std::chrono::system_clock::now();
r_vector = VectorFun<Double_v>( ran_vector,  N);   
auto endvector = std::chrono::system_clock::now();

std::chrono::duration<float,std::milli> durationvector = endvector - startvector;
std::cout << "Tiempo Vector " << durationvector.count() << " ms" << std::endl;
std::cout << "Suma Vector sum: " << r_vector << std::endl;


return 0;
}

Overwriting example_cuatro.cpp




Suma:





In [ ]:
%%bash
cd VecCore/install/lib/cmake/VecCore/
pwd




/content/VecCore/install/lib/cmake/VecCore


In [ ]:
%%bash
cd Vc/install
pwd


/content/Vc/install


In [ ]:
%%bash
cp /content/Taller_W_HPCSE-aceleracion/MacroUtilities.cmake /content/


In [ ]:
%%bash
cp /content/Taller_W_HPCSE-aceleracion/CMakeLists.txt /content/

In [ ]:
%%bash
cmake -DBACKEND=Vc -DVecCore_DIR=/content/VecCore/install/lib/cmake/VecCore \
-DCMAKE_BUILD_TYPE=Release -DVECCORE_ENABLE_VC=ON -DCMAKE_PREFIX_PATH=/content/Vc/install .

-- Configuring with Vc backend
-- Detected Compiler: GCC 7
-- Detected CPU: broadwell
-- Configuring done
-- Generating done
-- Build files have been written to: /content


CMake Deprecation Warning at Vc/install/lib/cmake/Vc/VcMacros.cmake:35 (cmake_minimum_required):
  Compatibility with CMake < 2.8.12 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.
Call Stack (most recent call first):
  Vc/install/lib/cmake/Vc/VcConfig.cmake:17 (include)
  /usr/local/lib/python3.7/dist-packages/cmake/data/share/cmake-3.22/Modules/CMakeFindDependencyMacro.cmake:47 (find_package)
  VecCore/install/lib/cmake/VecCore/VecCoreConfig.cmake:62 (find_dependency)
  CMakeLists.txt:61 (find_package)




In [ ]:
%%bash
make

Consolidate compiler generated dependencies of target returntypevec
[ 25%] Built target returntypevec
Consolidate compiler generated dependencies of target vectorcomplement
[ 37%] Building CXX object CMakeFiles/vectorcomplement.dir/example_dos.cpp.o
[ 50%] Linking CXX executable vectorcomplement
[ 50%] Built target vectorcomplement
Consolidate compiler generated dependencies of target vectorcartesian
[ 75%] Built target vectorcartesian
Consolidate compiler generated dependencies of target vectorsum
[ 87%] Building CXX object CMakeFiles/vectorsum.dir/example_cuatro.cpp.o
[100%] Linking CXX executable vectorsum
[100%] Built target vectorsum


/content/example_dos.cpp: In instantiation of ‘void ScalarGauss(int, float*) [with T = float]’:
/content/example_dos.cpp:153:61:   required from here
/content/example_dos.cpp:119:7: warning: format ‘%d’ expects argument of type ‘int’, but argument 2 has type ‘size_t {aka long unsigned int}’ [-Wformat=]
 printf("size %d \n", size);
 ~~~~~~^~~~~~~~~~~~~~~~~~~~
/content/example_dos.cpp: In instantiation of ‘void VectorGauss(int, float*) [with T = Vc_1::Vector<float, Vc_1::VectorAbi::Sse>]’:
/content/example_dos.cpp:160:63:   required from here
/content/example_dos.cpp:85:7: warning: format ‘%d’ expects argument of type ‘int’, but argument 2 has type ‘size_t {aka long unsigned int}’ [-Wformat=]
 printf("size %d \n", size);
 ~~~~~~^~~~~~~~~~~~~~~~~~~~


In [ ]:
%%bash 
 ./returntypevec 


Primer ejemplo 
size 1 
Acumulador 508.770935 
0.014419 ms
size 4 
Acumulador 508.770935 
0.00304 ms


In [ ]:
%%bash 
./vectorcomplement

main dos 
size 1 
41.5489 ms
size 4 
12.5614 ms
Histograma Distribucion Gaussiana Scalar
paso 0.571429 
bins 273 
bins 1861 
bins 9027 
bins 32101 
bins 83578 
bins 156956 
bins 216419 
bins 216337 
bins 157208 
bins 82892 
bins 32286 
bins 8964 
bins 1798 
bins 234 
Intervalo (-4.000000 a -3.428571 )  
Intervalo (-3.428571 a -2.857143 )  
Intervalo (-2.857143 a -2.285714 )  
Intervalo (-2.285714 a -1.714286 ) *** 
Intervalo (-1.714286 a -1.142857 ) ******** 
Intervalo (-1.142857 a -0.571428 ) *************** 
Intervalo (-0.571428 a 0.000000 ) ********************* 
Intervalo (0.000000 a 0.571429 ) ********************* 
Intervalo (0.571429 a 1.142858 ) *************** 
Intervalo (1.142858 a 1.714286 ) ******** 
Intervalo (1.714286 a 2.285715 ) *** 
Intervalo (2.285715 a 2.857143 )  
Intervalo (2.857143 a 3.428572 )  
Intervalo (3.428572 a 4.000000 )  
Histograma Distribucion Gaussiana Vector
paso 0.571429 
bins 252 
bins 1952 
bins 9200 
bins 31552 
bins 84000 
bins 156728 
bins 21549

In [ ]:
%%bash 
./vectorcartesian

Ejemplo tres 
size 1 
2.63938 ms
size 4 
2.26994 ms
rho scalar 76563.406250 theta scalar 78465.820312 rho vector 76563.406250 theta vector 78465.820312 


In [ ]:
%%bash 
./vectorsum

Ejemplo cuatro 
Tiempo Scalar 7000.15 ms
Suma Scalar  sum: 428904
Tiempo Vector 5511.21 ms
Suma Vector sum: 428904





Intervalo regula puede exactitud

Gracias

Impartido por Oscar Roberto Chaparro Amaro

<img src="https://github.com/OscarHuesos/Taller_W_HPCSE-aceleracion/blob/main/cic.jpeg?raw=true:, width=150" alt="simd" width=150>